In [1]:
### Análise Vendas

In [2]:
from pyspark.sql import SparkSession
import json
import os

In [3]:
#Definindo SPARK_LOCAL_IP antes de criar a SparkSession
os.environ["SPARK_LOCAL_IP"] = "10.2.1.27"  
os.environ["SPARK_HOME"]="/opt/spark"
#os.environ["PATH"]="$PATH:$SPARK_HOME/bin"


In [4]:
spark= SparkSession.builder.appName("Análise Vendas").getOrCreate() 
scRdd = spark.sparkContext.textFile("TRAB2/train.csv")
scDf = spark.read.option("header","True").csv("TRAB2/train.csv") #Lendo o arquivo como o DataFrame
scDf.persist()
scDf.is_cached



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 09:42:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


True

In [5]:
scDf.show(truncate=False) #exibindo dataframe
#sc.collect()


+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age  |Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001|P00069042 |F     |0-17 |10        |A            |2                         |0             |3                 |NULL              |NULL              |8370    |
|1000001|P00248942 |F     |0-17 |10        |A            |2                         |0             |1                 |6                 |14                |15200   |
|1000001|P00087842 |F     |0-17 |10        |A            |2                         |0             |12                |NULL              |NULL              |1422    

In [6]:
#Apresente o esquema da estrutura importad
scDf.printSchema()


root
 |-- User_ID: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Product_Category_1: string (nullable = true)
 |-- Product_Category_2: string (nullable = true)
 |-- Product_Category_3: string (nullable = true)
 |-- Purchase: string (nullable = true)



In [7]:
#Verifique a quantidade de linhas da estrutura importada
scDf.count()

550068

In [8]:
#Faça uma análise estatística descritiva básica(média, desvio padrão, quantidade, min,max ) para a coluna Purchase (transforme a coluna purchase para o tipo de dados inteiro)
from pyspark.sql.functions import col
colunaPurchase=scDf.withColumn("Purchase", col("Purchase").cast("int"))
colunaPurchase.printSchema()



root
 |-- User_ID: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Product_Category_1: string (nullable = true)
 |-- Product_Category_2: string (nullable = true)
 |-- Product_Category_3: string (nullable = true)
 |-- Purchase: integer (nullable = true)



In [9]:
colunaPurchase.select("Purchase").describe().show()

+-------+-----------------+
|summary|         Purchase|
+-------+-----------------+
|  count|           550068|
|   mean|9263.968712959126|
| stddev|5023.065393820554|
|    min|               12|
|    max|            23961|
+-------+-----------------+



In [10]:
#Verifique o número de produtos distintos contidos nesta base de dados
scDf.select("User_ID").distinct().count()

5891

In [11]:
#Verifique o número de pessoas que compraram os produtos por idade cruzados por gênero
cros=scDf.crosstab("Age","Gender")
crosstab_sort = cros.orderBy(col("Age_Gender").asc())  # Ordenando a tabela cross
crosstab_sort.show()


+----------+-----+------+
|Age_Gender|    F|     M|
+----------+-----+------+
|      0-17| 5083| 10019|
|     18-25|24628| 75032|
|     26-35|50752|168835|
|     36-45|27170| 82843|
|     46-50|13199| 32502|
|     51-55| 9894| 28607|
|       55+| 5083| 16421|
+----------+-----+------+



In [12]:
#Verifique o número de pessoas que compraram os produtos por idade cruzados por gênero
scDf_maior_vinteMil = scDf.filter(scDf.Purchase > 20000)
scDf_maior_vinteMil.count()

12691

In [17]:
#Crie outro data frame em que o valor da compras(purchase) foi maior que 20.000 e menor que 45000 somente para as mulheres('F'). Depois exiba o número de linhas:

scDf_maior_vinteMil_menor_ = scDf_maior_vinteMil.filter((scDf_maior_vinteMil.Gender =='F') & (scDf_maior_vinteMil.Purchase < 45000))
scDf_maior_vinteMil_menor_.count()

2822

In [25]:
#Encontre a média de compras (purchase) de cada uma das faixas de idade Resultado(s):
media = scDf.groupby("Age").agg({'Purchase':'mean'})
media.show()


+-----+-----------------+
|  Age|    avg(Purchase)|
+-----+-----------------+
|18-25|9169.663606261289|
|26-35|9252.690632869888|
| 0-17|8933.464640444974|
|46-50|9208.625697468327|
|51-55|9534.808030960236|
|36-45|9331.350694917874|
|  55+|9336.280459449405|
+-----+-----------------+



In [27]:
#Encontre a média de compras (purchase) de cada uma por genero:
mediaGn = scDf.groupby("Gender").agg({'Purchase':'mean'})
mediaGn.show()

+------+-----------------+
|Gender|    avg(Purchase)|
+------+-----------------+
|     F|8734.565765155476|
|     M|9437.526040472265|
+------+-----------------+



In [39]:
#Faça uma simulação gerando um novo dataframe com a média de compras por idade aumentada em 10%:
resultado10 = media.withColumn("Pur_10_percent",col("avg(Purchase)")*1.10)
resultado10.show()



+-----+-----------------+------------------+
|  Age|    avg(Purchase)|    Pur_10_percent|
+-----+-----------------+------------------+
|18-25|9169.663606261289|10086.629966887418|
|26-35|9252.690632869888|10177.959696156877|
| 0-17|8933.464640444974| 9826.811104489472|
|46-50|9208.625697468327|10129.488267215162|
|51-55|9534.808030960236| 10488.28883405626|
|36-45|9331.350694917874|10264.485764409661|
|  55+|9336.280459449405|10269.908505394345|
+-----+-----------------+------------------+

